In [ ]:
"""
File Name: ex6_optimization.ipynb

Description: 
dataset: cycle_sum_filtered_cae_v1_t1

Author: junghwan lee
Email: jhrrlee@gmail.com
Date Created: 2023.10.15


In [2]:
import os
import sys
import numpy as np
import importlib
import matplotlib.pyplot as plt
from pathlib import Path
import xlsxwriter
from tqdm import tqdm
import optuna
import pandas as pd
import time
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sqlite3
from tqdm.keras import TqdmCallback
from keras.callbacks import ModelCheckpoint, EarlyStopping

/mnt/d/sandbox/optimal_rul/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-04 20:29:30.505516: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-04 20:29:30.628869: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-04 20:29:37.228250: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warn

Usage

In [4]:
sys.path.append('../')
sys.path.append('../../dnn_models/')
gen_path = Path('..')/'..'/'resource'/ 'dataset'

proj_dir = "/mnt/d/sandbox/optimal_rul"  # Change this to your actual project path
sys.path.append(proj_dir)


In [5]:
import prediction_evaluation as PE
from prediction_evaluation import VisualEvaluation
import resnet_cbc as rsnet
from resnet_cbc import gen_resnet152_model, gen_resnet101_model, gen_resnet50_model, gen_resnet34_model, gen_resnet18_model
import hype_optimizer_cbc as HO
from hype_optimizer_cbc import HypeOptimizer, HypeOptimizer_v2
import data_distribution as DD
importlib.reload(rsnet)
importlib.reload(DD)
importlib.reload(PE)
importlib.reload(HO)

<module 'hype_optimizer_cbc' from '/mnt/d/sandbox/optimal_rul/optimization_v2/ex0/../hype_optimizer_cbc.py'>

In [6]:
battery_ids  = np.load(gen_path / 'battery_ids.npy')
eol = np.load(gen_path / 'eol_21_120.npy')
#cycle_data = np.load(gen_path / 'interpolated_cycle_data_21_120.npy')
new_cycle_sum = np.load(gen_path / 'new_cycle_sum_21_120.npy')

In [7]:
num_cells = eol.shape[0]
num_cycles = new_cycle_sum.shape[2]

# Repeat the EOL values for each cycle in the cycle_count array
eol_repeated = np.tile(eol, (1, num_cycles))
cycle_count = np.arange(1, num_cycles+1)
# Repeat the cycle_count values for each cell in the cycle_count array
cycle_count_repeated = np.tile(cycle_count, (num_cells, 1))
RUL = eol - num_cycles
RUL_repeated = eol_repeated - cycle_count_repeated

print("RUL shape:", RUL.shape)
print("RUL_repeated shape:", RUL_repeated.shape)
print("eol repeated shape:", eol_repeated.shape)
print("cycle_count shape:", cycle_count_repeated.shape)
print("eol shape:", eol.shape)
print("cycle_count shape:", cycle_count_repeated.shape)
print("new_cycle_sum shape:", new_cycle_sum.shape)

RUL shape: (124, 1)
RUL_repeated shape: (124, 100)
eol repeated shape: (124, 100)
cycle_count shape: (124, 100)
eol shape: (124, 1)
cycle_count shape: (124, 100)
new_cycle_sum shape: (124, 14, 100)


In [8]:
sorted_eol = np.sort(eol, axis=0)
sorted_eol_values = np.unique(sorted_eol, return_counts=True)
print(sorted_eol_values)

(array([ 148.,  300.,  335.,  429.,  438.,  444.,  457.,  458.,  459.,
        461.,  462.,  463.,  465.,  466.,  468.,  471.,  477.,  478.,
        480.,  481.,  483.,  485.,  487.,  489.,  492.,  494.,  495.,
        498.,  499.,  502.,  509.,  511.,  513.,  519.,  520.,  527.,
        534.,  535.,  541.,  559.,  561.,  599.,  616.,  617.,  625.,
        636.,  648.,  651.,  667.,  691.,  702.,  703.,  704.,  709.,
        713.,  719.,  731.,  742.,  757.,  772.,  786.,  788.,  796.,
        813.,  816.,  817.,  825.,  828.,  842.,  850.,  854.,  857.,
        858.,  860.,  862.,  870.,  876.,  880.,  917.,  923.,  932.,
        935.,  940.,  966.,  989., 1002., 1009., 1014., 1017., 1028.,
       1039., 1048., 1051., 1054., 1063., 1074., 1078., 1093., 1115.,
       1146., 1155., 1156., 1158., 1284., 1315., 1434., 1638., 1709.,
       1801., 1836., 1852., 1935., 2160., 2237.]), array([1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 2, 1, 1, 1

In [9]:
# Calculate quantile values
quantiles = [np.percentile(sorted_eol, q=i) for i in range(10, 100, 10)]
# Print quantiles
print(quantiles)
# You can use these quantiles as bin edges for stratification
bins = [-np.inf] + quantiles + [np.inf]
print(bins)

[462.3, 487.0, 518.4, 638.4000000000001, 736.5, 828.0, 876.4, 1021.4000000000001, 1155.7]
[-inf, 462.3, 487.0, 518.4, 638.4000000000001, 736.5, 828.0, 876.4, 1021.4000000000001, 1155.7, inf]


In [10]:
def generate_bins(sorted_eol, train_percent=0.8):
    N = len(sorted_eol)
    min_samples = int(N / train_percent * (1 - train_percent))
    
    unique_values, counts = np.unique(sorted_eol, return_counts=True)
    
    bins = [float('-inf')]
    running_total = 0

    for val, count in zip(unique_values, counts):
        running_total += count
        if running_total >= min_samples:
            bins.append(val)
            running_total = 0

    bins.append(float('inf'))
    
    return bins
    

In [11]:
bins = generate_bins(sorted_eol, 0.8)

In [12]:
bins

[-inf, 495.0, 719.0, 923.0, 1836.0, inf]

In [13]:
# no. of cell, no. of cycles, no. of features
transposed_cycle_sum = np.transpose(new_cycle_sum, axes=(0, 2, 1))

# split data
(X_train,
 X_val,
 y_train,
 y_val,
 indices_train,
 indices_val) = DD.data_distribution(transposed_cycle_sum, RUL_repeated, eol_repeated, 0.8, bins)

# for cycle by cycle, samples = no. of cell * no. of cycles
X_train = X_train.reshape(-1, X_train.shape[-1])
X_val = X_val.reshape(-1, X_val.shape[-1])
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

# generate scalers using training data.
X_scaler, y_scaler = DD.fit_scalers(X_train, y_train)
X_train_norm, y_train_norm = DD.apply_normalization(X_train, y_train, X_scaler, y_scaler)
X_val_norm, y_val_norm = DD.apply_normalization(X_val, y_val, X_scaler, y_scaler)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print(X_train_norm.shape, X_val_norm.shape, y_train_norm.shape, y_val_norm.shape)
print('indices_train:', indices_train)
print('indices_val:', indices_val)
DD.save_scalers(X_scaler, y_scaler)

(9900, 14) (2500, 14) (9900, 1) (2500, 1)
(9900, 14) (2500, 14) (9900, 1) (2500, 1)
indices_train: [  6  55  53  90  88  82  75  33  26  19  18  96   4  99  11  74  54  79
  22  15  12  17  36  81  69   9   0  20  48 113  86  41  56  40  57 101
  98 107  85  51  73   1 123  21  71  34  89  92  25 109  62   2  76 112
  61  70  59  14   7  60  43  93  50  37  52  30 118  16  64  67  97 106
 104  78 116 122 115  49  35  80  68  65  83 119 120 114  95  13 110  10
 100 103  63   8  44  87   3  94 121]
indices_val: [ 77  24  32  58  38  42  28 105   5  45  27  66  91  29  47 102  72 117
  23  31  84  39 108  46 111]
X_scaler saved as X_scaler.pkl
y_scaler saved as y_scaler.pkl


In [14]:
# Reshape the input data to the channel dimension
X_train_norm = X_train_norm.reshape(X_train_norm.shape[0], X_train_norm.shape[1], 1)
X_val_norm = X_val_norm.reshape(X_val_norm.shape[0], X_val_norm.shape[1], 1)

print(X_train_norm.shape, X_val_norm.shape)

(9900, 14, 1) (2500, 14, 1)


In [15]:
from pathlib import Path  # Ensure this is imported

grid_search_space = {
    'batch_size': [1, 2, 4, 8, 16],
    'batch_norm': [1, 0]
}

custom_objects = {
    "mean_absolute_percentage_error": HO.mean_absolute_percentage_error,
    "root_mean_squared_error": HO.root_mean_squared_error
}

def hype_space(trial, input_shape):
    # Use values directly from grid_search_space
    batch_size = trial.suggest_categorical('batch_size', grid_search_space['batch_size'])
    batch_norm = trial.suggest_categorical('batch_norm', grid_search_space['batch_norm'])

    use_bn = True if batch_norm == 1 else False  # Fix logic

    model = gen_resnet34_model(
        input_shape, 
        kernel_size=3,
        strides=2,
        activation='leaky_relu', 
        learning_rate=0.0001, 
        loss='mse',
        metrics=['mean_absolute_error', 'mean_squared_error', HO.mean_absolute_percentage_error, HO.root_mean_squared_error],
        use_bn=use_bn
    )

    return model, batch_size

In [16]:
# for test handling kernel crash
db_name = 'resnet_34_ex0.db'
study_name = 'resnet_34_ex0'
n_trial = 10
base_path = Path('./training_data/')


sampler = optuna.samplers.GridSampler(grid_search_space)
optimizer = HypeOptimizer_v2(sampler, hype_space, custom_objects, n_trial, base_path, db_name, study_name, direction='minimize')


Study Folder 'training_data/resnet_34_ex0' already exists!
DB Folder 'training_data/db' already exists!


[I 2025-03-04 20:29:52,567] Using an existing study with name 'resnet_34_ex0' instead of creating a new one.


In [25]:
optimizer.run(X_train_norm, y_train_norm, X_val_norm, y_val_norm, y_scaler, y_train, y_val, max_epoch=500, patience=50)

traial:  0
epoch_file_name:  training_data/resnet_34_ex0/epoch_info/epoch_info_0.txt
model_name:  training_data/resnet_34_ex0/models/resnet_34_ex0_0.keras
savefile_name:  training_data/resnet_34_ex0/results/resnet_34_ex0_results_0.xlsx
🚀 No previous model found. Initializing a new model.


2025-03-02 13:02:15.326107: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-02 13:02:15.326194: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-02 13:02:15.326231: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-02 13:02:15.326253: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-02 13:02:15.326272: I external/local_xla/xla/stream_executor

trial: <optuna.trial._trial.Trial object at 0x7f5242428ca0> training_data/resnet_34_ex0/epoch_info/epoch_info_0.txt


  0%|          | 0/500 [00:00<?, ?epoch/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1740888139.570713    9751 service.cc:145] XLA service 0x7f5078004880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740888139.570757    9751 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
I0000 00:00:1740888139.570761    9751 service.cc:153]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-03-02 13:02:20.193181: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-02 13:02:21.269833: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1740888145.211891    9751 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetim

310/310 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Training set metrics:
RMSE: 60.5327206450701
MAE: 34.06535587503453
MAPE: 0.04300072704472321
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Validation set metrics:
RMSE: 118.24175237141561
MAE: 77.15064681396484
MAPE: 0.24881876399114147


...

 29%|██▉       | 144/500 [1:24:02<3:27:45, 35.02s/epoch, loss=0.00304, mean_absolute_error=0.0411, mean_absolute_percentage_error=0.224, mean_squared_error=0.00304, root_mean_squared_error=0.0491, val_loss=4.87, val_mean_absolute_error=0.904, val_mean_absolute_percentage_error=3.22, val_mean_squared_error=4.87, val_root_mean_squared_error=0.921]         


310/310 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step
Training set metrics:
RMSE: 154.11284855356297
MAE: 105.36630268246236
MAPE: 0.1708429474640636
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Validation set metrics:
RMSE: 150.51580948807893
MAE: 104.85192560119629
MAPE: 0.31552122141379213


[I 2025-03-02 19:28:17,370] Trial 9 finished with value: 0.24318208443892786 and parameters: {'batch_size': 4, 'batch_norm': 1}. Best is trial 3 with value: 0.12707361540270246.


Best Hyperparameters: {'batch_size': 16, 'batch_norm': 0}
Best score: 0.12707361540270246


number          MAPE  params_batch_norm  params_batch_size     state  \
0       0  1.459097e-01                  0                  4  COMPLETE   
1       1  1.490477e+24                  1                  1  COMPLETE   
2       2  1.724707e-01                  1                 16  COMPLETE   
3       3  1.270736e-01                  0                 16  COMPLETE   
4       4  1.419666e-01                  0                  1  COMPLETE   
5       5  1.820355e-01                  0                  2  COMPLETE   
6       6  1.359183e-01                  0                  8  COMPLETE   
7       7  2.957246e+00                  1                  2  COMPLETE   
8       8  1.846690e-01                  1                  8  COMPLETE   
9       9  2.431821e-01                  1                  4  COMPLETE   

   Prediction Time (ms)  user_attrs_train_mae_rul  user_attrs_train_mape_rul  \
0           2775.305271              3.406536e+01               4.300073e-02   
1           3200.896263              7.732146e+26               1.219707e+24   
2           2909.952641              8.038008e+01               1.211564e-01   
3           1707.912683              2.574487e+01               3.558290e-02   
4           1793.318987              2.414245e+01               3.712122e-02   
5           1752.321005              8.556706e+01               1.140784e-01   
6           1732.304335              4.232230e+01               5.455979e-02   
7           2952.186108              1.231190e+03               2.252868e+00   
8           3043.015718              7.724413e+01               1.163507e-01   
9           3284.706354              1.053663e+02               1.708429e-01   

   user_attrs_train_rmse_rul  user_attrs_training_time  \
0               6.053272e+01               1733.455412   
1               8.332816e+26               4399.329757   
2               1.089177e+02                807.625724   
3               3.840055e+01                606.626238   
4               3.287237e+01               4476.129683   
5               1.285658e+02               1766.561131   
6               6.024407e+01                622.469679   
7               1.542525e+03               2351.784141   
8               9.453478e+01               1298.539398   
9               1.541128e+02               5042.270191   

   user_attrs_val_mae_rul  user_attrs_val_mape_rul  user_attrs_val_rmse_rul  
0            7.715065e+01             2.488188e-01             1.182418e+02  
1            7.403032e+26             1.761246e+24             8.050448e+26  
2            9.043460e+01             2.237850e-01             1.158790e+02  
3            8.535544e+01             2.185643e-01             1.174339e+02  
4            7.629716e+01             2.468120e-01             1.176954e+02  
5            8.886356e+01             2.499926e-01             1.103905e+02  
6            7.672551e+01             2.172769e-01             1.058403e+02  
7            1.543659e+03             3.661625e+00             1.877774e+03  
8            8.433431e+01             2.529873e-01             1.211295e+02  
9            1.048519e+02             3.155212e-01             1.505158e+02

In [17]:
df = optimizer.get_trial_results()

In [18]:
df.to_csv(base_path / study_name /'trials.csv', index=False)

In [19]:
df

number          MAPE  params_batch_norm  params_batch_size     state  \
0       0  1.459097e-01                  0                  4  COMPLETE   
1       1  1.490477e+24                  1                  1  COMPLETE   
2       2  1.724707e-01                  1                 16  COMPLETE   
3       3  1.270736e-01                  0                 16  COMPLETE   
4       4  1.419666e-01                  0                  1  COMPLETE   
5       5  1.820355e-01                  0                  2  COMPLETE   
6       6  1.359183e-01                  0                  8  COMPLETE   
7       7  2.957246e+00                  1                  2  COMPLETE   
8       8  1.846690e-01                  1                  8  COMPLETE   
9       9  2.431821e-01                  1                  4  COMPLETE   

   Prediction Time (ms)  user_attrs_train_mae_rul  user_attrs_train_mape_rul  \
0           2775.305271              3.406536e+01               4.300073e-02   
1           3200.896263              7.732146e+26               1.219707e+24   
2           2909.952641              8.038008e+01               1.211564e-01   
3           1707.912683              2.574487e+01               3.558290e-02   
4           1793.318987              2.414245e+01               3.712122e-02   
5           1752.321005              8.556706e+01               1.140784e-01   
6           1732.304335              4.232230e+01               5.455979e-02   
7           2952.186108              1.231190e+03               2.252868e+00   
8           3043.015718              7.724413e+01               1.163507e-01   
9           3284.706354              1.053663e+02               1.708429e-01   

   user_attrs_train_rmse_rul  user_attrs_training_time  \
0               6.053272e+01               1733.455412   
1               8.332816e+26               4399.329757   
2               1.089177e+02                807.625724   
3               3.840055e+01                606.626238   
4               3.287237e+01               4476.129683   
5               1.285658e+02               1766.561131   
6               6.024407e+01                622.469679   
7               1.542525e+03               2351.784141   
8               9.453478e+01               1298.539398   
9               1.541128e+02               5042.270191   

   user_attrs_val_mae_rul  user_attrs_val_mape_rul  user_attrs_val_rmse_rul  
0            7.715065e+01             2.488188e-01             1.182418e+02  
1            7.403032e+26             1.761246e+24             8.050448e+26  
2            9.043460e+01             2.237850e-01             1.158790e+02  
3            8.535544e+01             2.185643e-01             1.174339e+02  
4            7.629716e+01             2.468120e-01             1.176954e+02  
5            8.886356e+01             2.499926e-01             1.103905e+02  
6            7.672551e+01             2.172769e-01             1.058403e+02  
7            1.543659e+03             3.661625e+00             1.877774e+03  
8            8.433431e+01             2.529873e-01             1.211295e+02  
9            1.048519e+02             3.155212e-01             1.505158e+02

In [20]:
trained_model = optimizer.get_trained_model(3)

2025-03-04 20:30:00.528814: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-04 20:30:00.528914: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-04 20:30:00.528949: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-04 20:30:00.528970: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-04 20:30:00.528989: I external/local_xla/xla/stream_executor

In [20]:
trained_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 14, 1)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 7, 64)        512         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 7, 64)        0           conv1d[0][0]                     
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 4, 64)        0           activation[0][0]                 
______________________________________________________________________________________________

In [ ]:
for layer in trained_model.layers:
    weights = layer.get_weights()
    print(weights)

In [21]:
X_scaler, y_scaler = DD.load_scalers()

In [22]:
transposed_cycle_sum.shape

(124, 100, 14)

In [23]:
reshaped_cycle_sum = transposed_cycle_sum.reshape(-1, transposed_cycle_sum.shape[-1])
reshaped_rul = RUL_repeated.reshape(-1, 1)
X_test_norm, y_test_norm = DD.apply_normalization(reshaped_cycle_sum, reshaped_rul, X_scaler, y_scaler)

In [24]:
# evaluation using normalied data.
loss, mae, mse, mape, rmse = trained_model.evaluate(X_test_norm, y_test_norm)
print(f"Loss: {loss}, Mean Absolute Error: {mae}, Mean Squared Error: {mse}, Mean Absolute Percentage Error: {mape}, Root Mean Squared Error: {rmse}")

I0000 00:00:1741087810.311129  282913 service.cc:145] XLA service 0x7f5aac0286a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741087810.311244  282913 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
I0000 00:00:1741087810.311248  282913 service.cc:153]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-03-04 20:30:10.374761: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-03-04 20:30:10.445767: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 45/388 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0010 - mean_absolute_error: 0.0229 - mean_absolute_percentage_error: 0.0533 - mean_squared_error: 0.0010 - root_mean_squared_error: 0.0256        

I0000 00:00:1741087811.560685  282913 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0011 - mean_absolute_error: 0.0209 - mean_absolute_percentage_error: 0.1351 - mean_squared_error: 0.0011 - root_mean_squared_error: 0.0232
Loss: 0.0009547426598146558, Mean Absolute Error: 0.018547704443335533, Mean Squared Error: 0.0009547426598146558, Mean Absolute Percentage Error: 0.1285049468278885, Root Mean Squared Error: 0.02074107713997364


In [25]:
# unnormalized x: (, 14)
ruls = PE.RUL_predict(trained_model, reshaped_cycle_sum, X_scaler, y_scaler)

388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [26]:
print(ruls)

[[1794]
 [1802]
 [1770]
 ...
 [1715]
 [1724]
 [1727]]


In [27]:
results = PE.evaluate_predictions(reshaped_rul, ruls)
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

MSE: 3957.3204
RMSE: 62.9072
MAE: 37.7614
MAPE: 7.2469


/mnt/d/sandbox/optimal_rul/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


usage of Visual evaluation

In [53]:
import prediction_evaluation
importlib.reload(prediction_evaluation)
from prediction_evaluation import VisualEvaluation


In [54]:
vev = VisualEvaluation(reshaped_rul, ruls, transposed_cycle_sum.shape[0], transposed_cycle_sum.shape[1])

/mnt/d/sandbox/optimal_rul/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/mnt/d/sandbox/optimal_rul/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/mnt/d/sandbox/optimal_rul/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/mnt/d/sandbox/optimal_rul/.venv/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 an

In [52]:
vev.heatmap(title = 'Predicted RULs', cmap_color = 'RdBu_r', title_fontsize=20, label_fontsize=20, tick_fontsize=20, 
                 cbar_fontsize=20, dpi=300, max_xticks=10, max_yticks=20)


In [55]:
vev.box(None, width = 30, height = 3)

/mnt/d/sandbox/optimal_rul/optimization_v2/ex0/../prediction_evaluation.py:581: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  plt.title(title, fontsize=20)
